# Research Track II - Enrico Fiasché
# User Interface
## Initialize node

In [1]:
import rospy
import jupyros as jr
import time
from rt2_assignment1.srv import Command
import actionlib
import rt2_assignment1.msg
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry

rospy.init_node('user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)

# action client used to cancel the goal
action_client = actionlib.SimpleActionClient("go_to_point",rt2_assignment1.msg.PositionAction)
if(action_client.wait_for_server()):
    print("Connected!")
else:
    print("Unable to connect with the action server")

twist_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

Connected!


## Start and Stop random targets

In [2]:
from std_msgs.msg import Int32, Bool

start_sim = Button(description='START',
            layout=Layout(width='200', align="center", grid_area='b1'),
            style=ButtonStyle(button_color='lightgreen'))
stop_sim = Button(description='STOP',
            layout=Layout(width='200', align="center", grid_area='b2'),
            style=ButtonStyle(button_color='salmon'))

HBox([start_sim,stop_sim])

grid = GridBox(children=[start_sim, stop_sim],
        layout=Layout(
        width='40%',
        grid_template_rows='auto auto',
        grid_template_columns='33% 15% 33%',
        grid_template_areas=''' "b1 . b2 " ''')
        )

display(grid)
starting_time = 0

def start(event):
    global starting_time
    
    ui_client("start")
    starting_time = rospy.Time.now()
    
def stop(event):
    action_client.cancel_all_goals()
    ui_client("stop")
    
start_sim.on_click(start)
stop_sim.on_click(stop)

GridBox(children=(Button(description='START', layout=Layout(grid_area='b1', width='200'), style=ButtonStyle(bu…

We did it
We did it ['Reached', 'Cancelled'] [0, 1]
We did it ['Reached', 'Cancelled'] [1, 2]
We did it ['Reached', 'Cancelled'] [1, 3]
We did it ['Reached', 'Cancelled'] [0, 1]
We did it ['Reached', 'Cancelled'] [0, 2]
We did it ['Reached', 'Cancelled'] [1, 3]
We did it ['Reached', 'Cancelled'] [1, 4]


## Sliders

In [3]:
import ipywidgets as widgets

linear_slider = widgets.FloatSlider(value=0.2, min=0, max=0.5, step=0.1, description='linear_velocity')
angular_slider = widgets.FloatSlider(value=-3.0, min=-5.0, max=0, step=0.5, description='angular_velocity')
lin_vel = linear_slider.value
ang_vel = angular_slider.value

def on_linear_change(change):
    global lin_vel, ang_vel
    
    const_twist = Twist()
    lin_vel = change['new']
    const_twist.linear.x = lin_vel
    const_twist.angular.z = ang_vel
    pub_consts.publish(const_twist)
    
def on_angular_change(change):
    global lin_vel, ang_vel
    
    const_twist = Twist()
    ang_vel = change['new']
    const_twist.linear.x = lin_vel
    const_twist.angular.z = ang_vel
    pub_consts.publish(const_twist)

pub_consts = rospy.Publisher('/consts', Twist, queue_size=1)
linear_slider.observe(on_linear_change, names='value')
angular_slider.observe(on_angular_change, names='value')

widgets.HBox([widgets.VBox([linear_slider, angular_slider])])

## Controller

In [4]:
up_arrow = Button(description='^',
            layout=Layout(width='auto', align="center", grid_area='b1'),
            style=ButtonStyle(button_color='lightgreen'))
down_arrow = Button(description='v',
            layout=Layout(width='auto', align="center", grid_area='b5'),
            style=ButtonStyle(button_color='lightgreen'))
left_arrow = Button(description='<',
            layout=Layout(width='auto', align="center", grid_area='b2'),
            style=ButtonStyle(button_color='lightgreen'))
right_arrow = Button(description='>',
            layout=Layout(width='auto', align="center", grid_area='b4'),
            style=ButtonStyle(button_color='lightgreen'))
stopRobot_button = Button(description='STOP',
            layout=Layout(width='auto', align="center", grid_area='b3'),
            style=ButtonStyle(button_color='salmon'))

controller_grid = GridBox(children=[up_arrow,down_arrow,left_arrow,right_arrow,stopRobot_button],
        layout=Layout(
        width='40%',
        grid_template_rows='auto auto',
        grid_template_columns='25% 10% 20% 20% 20%',
        grid_template_areas='''
        ". . . b1 . "
        "b3 . b2 b5 b4"
        ''')
        )

def up(event):
    global lin_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = lin_vel
    twist_msg.angular.z = 0.0
    twist_pub.publish(twist_msg)

def down(event):
    global lin_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = -lin_vel
    twist_msg.angular.z = 0.0
    twist_pub.publish(twist_msg)

def left(event):
    global ang_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = ang_vel
    twist_pub.publish(twist_msg)
    
def right(event):
    global ang_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = -ang_vel
    twist_pub.publish(twist_msg)
    
def stopRobot(event):
    global lin_vel, ang_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = 0.0
    twist_pub.publish(twist_msg)
    
display(controller_grid)

up_arrow.on_click(up)
down_arrow.on_click(down)
left_arrow.on_click(left)
right_arrow.on_click(right)
stopRobot_button.on_click(stopRobot)

GridBox(children=(Button(description='^', layout=Layout(grid_area='b1', width='auto'), style=ButtonStyle(butto…

# Graphical Information

## Cmd_vel vs Actual Velocity

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib widget

fig_vel = plt.figure(1)
ax_cmd = fig_vel.add_subplot(211)
ax_odom = fig_vel.add_subplot(212)
"""
fig_vel, (ax_cmd, ax_odom) = plt.subplots(2)
plot_cmd_lin, = ax_cmd.plot([], [])
plot_cmd_ang, = ax_cmd.plot([], [])
plot_odom_lin, = ax_odom.plot([], [])
plot_odom_ang, = ax_odom.plot([], [])
"""
plot_cmd_lin, = ax_cmd.plot([], [])
plot_cmd_ang, = ax_odom.plot([], [])
plot_odom_lin, = ax_cmd.plot([], [])
plot_odom_ang, = ax_odom.plot([], [])

plots = [plot_cmd_lin, plot_cmd_ang, plot_odom_lin, plot_odom_ang]

ax_cmd.set(xlabel='Time', ylabel='Amplitude')
ax_odom.set(xlabel='Time', ylabel='Amplitude')

time_cmd, time_odom, lin_cmd, ang_cmd, lin_odom, ang_odom = [], [], [], [], [], []


def cmd_callback(cmd_msg):
    global lin_cmd, ang_cmd, time_cmd
    
    lin_cmd.append(cmd_msg.linear.x)
    ang_cmd.append(cmd_msg.angular.z)

    if(len(time_cmd) == 0):
        time_cmd.append(0.05)
    else:
        time_cmd.append(time_cmd[len(time_cmd)-1]+0.05)

def odom_callback_velocity(odom_msg):
    global lin_odom, ang_odom, time_odom
    
    lin_odom.append(odom_msg.twist.twist.linear.x)
    ang_odom.append(odom_msg.twist.twist.angular.z)
    
    if(len(time_odom) == 0):
        time_odom.append(0.05)
    else:
        time_odom.append(time_odom[len(time_odom)-1]+0.05)
    
def plot_init():
   
    ax_cmd.set_xlim(0,5)
    ax_cmd.set_ylim(-1.0,1.0)
    
    ax_odom.set_xlim(0,5)
    ax_odom.set_ylim(-1.0,1.0)

    return plots,

def update_plot(frame):

    xmin, xmax = ax_cmd.get_xlim()
    if time_cmd[len(time_cmd)-1] > xmax:
        ax_cmd.set_xlim(xmin,2*xmax)
        ax_cmd.figure.canvas.draw()

    xmin, xmax = ax_odom.get_xlim()
    if time_odom[len(time_odom)-1] > xmax:
        ax_odom.set_xlim(xmin,2*xmax)
        ax_odom.figure.canvas.draw()

    plot_cmd_lin.set_data(time_cmd, lin_cmd)
    plot_cmd_ang.set_data(time_cmd, ang_cmd)
    
    plot_odom_lin.set_data(time_odom, lin_odom)
    plot_odom_ang.set_data(time_odom, ang_odom)
    
    return plots,

jr.subscribe('/cmd_vel', Twist, cmd_callback)
jr.subscribe('/odom', Odometry, odom_callback_velocity)

ani = FuncAnimation(fig_vel, update_plot, init_func=plot_init)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now
[WARN] [1626729746.695783, 280.351000]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
Removing previous callback, only one redirection possible right now


## Plot targets reached and cancelled

In [8]:
from std_msgs.msg import Int32
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib widget

names = ["Reached","Cancelled"]
values = [0,0]
c = ["lightgreen","salmon"]

fig_targets = plt.figure(2) 
ax_targets = fig_targets.add_subplot(111)
bar_plot = ax_targets.bar(names,values, color=c)

plt.title('Number of target reached and cancelled')
plt.xlabel('Numbers')

def target_callback(msg):
    global values, bar_plot, ax_targets

    if(msg.data == 1):
        values[0] += 1

    elif(msg.data == 0):
        values[1] += 1
    
    bar_plot.remove()
    rospy.sleep(0.5)
    bar_plot = ax_targets.bar(names,values, color=c)
    print("We did it "+str(names)+" "+str(values))

jr.subscribe('stateResult', Int32, target_callback)
#.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Removing previous callback, only one redirection possible right now


## Time required to reach a target

In [6]:
import rospy # remember to delete it
from std_msgs.msg import Int32, Bool

def cb(msg):
    global starting_time
    
    if(msg.data == 1):
        finish_time = rospy.Time.now()
        with out:
            print("Time to reach = "+str(finish_time.to_sec() - starting_time.to_sec()))
        starting_time = rospy.Time.now()
        
out = widgets.Output()
jr.subscribe('stateResult', Int32, cb)

Removing previous callback, only one redirection possible right now


# Robot position and orientation

In [5]:
from nav_msgs.msg import Odometry
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tf.transformations import quaternion_matrix
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot()
#odom_fig, odom_ax = plt.subplots()
plot_odom, = plt.plot([], [], 'ro')
x_data, y_data = [] , []

def plot_init():
    #odom_ax.set_xlim(-5, 5)
    #odom_ax.set_ylim(-5, 5)
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    plot_odom.set_data([],[])
    
    return plot_odom,

def odom_callback(msg):
    global x_data, y_data
    
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)

def update_plot(frame):
    plot_odom.set_data(x_data, y_data) 
    return plot_odom,

rospy.Subscriber('/odom', Odometry, odom_callback)
ani = FuncAnimation(fig, update_plot, init_func=plot_init)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …